# Exercise - Enable Tool Calling - SOLUTION

In this exercise, you’ll enhance your AI agent by adding tool-calling capabilities, allowing it to interact with external functions dynamically.

**Challenge**

Imagine you're building an AI-powered assistant that helps users with various tasks such as:

- Fetching real-time stock prices
- Performing complex calculations
- Querying a weather API
- Searching a database

Instead of manually deciding when to call which function, your AI agent will automatically detect when a tool is needed and invoke it.

## 0. Import the necessary libs

In [ ]:
import requests
import datetime
import inspect
import json
from typing import (
    TypedDict, 
    List, Dict, Literal, 
    Callable, Optional, Any, 
    get_type_hints
)
from openai import OpenAI
from openai.types.chat.chat_completion_message import ChatCompletionMessage
from openai.types.chat.chat_completion_message_tool_call import ChatCompletionMessageToolCall
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("API_KEY")
endpoint = os.getenv("OPENAI_ENDPOINT")

client = OpenAI(
            api_key=api_key,
            base_url=endpoint
        )

## 1. Recap: how to use OpenAI client with your API Key

To be able to connect with OpenAI, you need to instantiate an OpenAI client passing your OpenAI key.


## 2. Recap: Memory & Function

Recently we combined memory with custom Python functions to create the full cycle of tool calling, which enabled an LLM to interact with the world.

In [ ]:
class Memory:
    def __init__(self):
        self._messages: List[Dict[str, str]] = []
    
    def add_message(self, 
                    role: Literal['user', 'system', 'assistant', 'tool'], 
                    content: str,
                    tool_calls: dict=dict(),
                    tool_call_id=None)-> None:

        message = {
            "role": role,
            "content": content,
            "tool_calls": tool_calls,
        }

        if role == "tool":
            message = {
                "role": role,
                "content": content,
                "tool_call_id": tool_call_id,
            }

        self._messages.append(message)

    def get_messages(self) -> List[Dict[str, str]]:
        return self._messages

    def last_message(self) -> None:
        if self._messages:
            return self._messages[-1]

    def reset(self) -> None:
        self._messages = []

In [ ]:
def chat_with_tools(user_question:str=None, 
                    memory:Memory=None, 
                    model:str="gpt-4o-mini", 
                    temperature=0.0, 
                    tools=None)-> str:
    messages = [{"role": "user", "content": user_question}]
    if memory:
        if user_question:
            memory.add_message(role="user", content=user_question)
        messages = memory.get_messages()        
    
    response = client.chat.completions.create(
        model = model,
        temperature = temperature,
        messages = messages,
        tools=tools,
    )
    
    ai_message = str(response.choices[0].message.content)
    tool_calls = response.choices[0].message.tool_calls
    
    if memory:
        memory.add_message(role="assistant", content=ai_message, tool_calls=tool_calls)
    
    return ai_message

In [ ]:
def power(base:float, exponent:float):
    """Exponentatiation: base to the power of exponent"""
    
    return base ** exponent

In [ ]:
tools = [{
    "type": "function",
    "function": {
        "name": "power",
        "description": "Exponentatiation: base to the power of exponent",
        "parameters": {
            "type": "object",
            "properties": {
                "base": {"type": "number"},
                "exponent": {"type": "number"}
            },
            "required": ["base", "exponent"],
            "additionalProperties": False
        },
        "strict": True
    }
}]

In [ ]:
# Instantiate memory and start with the system prompt
memory = Memory()
memory.add_message(role="system", content="You're a helpful assitant")

# Call the LLM with a question that needs a tool
ai_message = chat_with_tools(
    "2 to the power of -5?",
    tools=tools,
    memory=memory,
)

# Get the arguments from the tool_calls object and call the actual defined function
args = json.loads(memory.last_message()['tool_calls'][0].function.arguments)
result = power(args["base"], args["exponent"])

# Extract the tool_call_id and feed the LLM with the result from the function 
tool_call_id = memory.last_message()['tool_calls'][0].id
memory.add_message(role="tool", content=str(result), tool_call_id=tool_call_id)
ai_message = chat_with_tools(
    tools=tools,
    memory=memory,
)

In [ ]:
memory.get_messages()

## 3. Create Tool abstractions

Although powerful, the way we've built by calling manually is prone to errors. What if you don't pass the correct type or miss one required field in the json-schema?

Your task is creating an abstraction to make it easier to build a tool and call it. 

Tip: Inspect the json schema of the tool we created to help you.

Your class should have at least the following methods: 
- `__init__()` receiving the function and some logic to extract docs, arguments and their types
- `dict()` to return the json schema
- `__call__()` to enable the object instantiated to be callable. 

Example:
```python
class Tool:
    def __init__(self, func:Callable):
        self.func = func
    
    def dict(self):
        pass

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)  

def my_func(arg1:int)->str:
    return "ok"

my_tool = Tool(my_func)
my_tool(arg1=1)
```



It's important learn about the following Python methods to understand how to parse functions and get docstrings, arguments and types:
- typing.get_type_hints()
- inspect.signature()

In [ ]:
class Tool:
    def __init__(self, func:Callable):
        self.func = func
        self.name = func.__name__
        self.description = func.__doc__
        self.argument_types_map = get_type_hints(func)
        self.signature = inspect.signature(func)
        self.arguments = [
            {
                "name": key, 
                "type": self._infer_json_schema_type(value),
                "required": param.default == inspect.Parameter.empty
            } 
            for key, value in self.argument_types_map.items()
            if (param := self.signature.parameters.get(key))
        ]

    def dict(self):
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": self.description,
                "parallel_tool_calls": False,
                "parameters": {
                    "type": "object",
                    "properties": {
                        argument["name"]: {
                            "type": argument["type"],
                        }
                        for argument in self.arguments
                    },
                    "required": [
                        argument["name"] 
                        for argument in self.arguments 
                        if argument["required"]
                    ],
                    "additionalProperties": False,
                },
                "strict": True
            }
        }

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)
    
    def _infer_json_schema_type(self, arg_type: Any) -> str:
        if arg_type == bool:
            return "boolean"
        elif arg_type == int:
            return "integer"
        elif arg_type == float:
            return "number"
        elif arg_type == str:
            return "string"
        elif arg_type == list:
            return "array"
        elif arg_type == dict:
            return "object"
        elif arg_type is None:
            return "null"
        elif arg_type == datetime.date or arg_type == datetime.datetime:
            return "string"  # JSON Schema treats dates as strings
        else:
            return "string"  # Default to string if type is unknown

In [ ]:
power_tool = Tool(power)
print(power_tool.dict())
print(power_tool(2,3))

In [ ]:
# Now using the Tool Abstraction:

# Instantiate memory and start with the system prompt
memory = Memory()
memory.add_message(role="system", content="You're a helpful assitant")

# Call the LLM with a question that needs a tool
ai_message = chat_with_tools(
    "2 to the power of -5?",
    tools=list().append(power_tool.dict()),
    memory=memory,
)

# Get the arguments from the tool_calls object and call the actual defined function
result = power_tool(2,-5)

# Extract the tool_call_id and feed the LLM with the result from the function 
tool_call_id = memory.last_message()['tool_calls'][0].id
memory.add_message(role="tool", content=str(result), tool_call_id=tool_call_id)
ai_message = chat_with_tools(
    tools=tools,
    memory=memory,
)

## 3. Update the Agent class

You will enhance the logic of your agent so that it can handle user queries and interact with external tools dynamically. The goal is to refine how the agent processes user messages, generates responses, and invokes tools when necessary.

**Objective**

You will modify the logic responsible for:

- Processing user input – The agent should record and manage conversation history.
- Generating a response – The agent will use a language model to create a reply based on previous messages.
- Identifying when tools are needed – If a tool is required to complete the request, the agent should detect this and trigger the appropriate function.
- Handling tool execution and responses – The agent should execute the tool, capture its output, and integrate the result into the conversation.

**Steps**

- Update the logic to check whether a tool needs to be invoked based on the AI-generated response.
- If tools are needed, execute them with the correct arguments.
- Incorporate tool results into the conversation so that the AI can refine its response using the additional information.
- Ensure the agent can handle multiple tool calls recursively, meaning if the response suggests using another tool after the first one, it should be processed correctly.

**Considerations**

- Think about how the agent decides when to call a tool.
- Ensure the agent stores the tool’s response correctly so it can continue the conversation naturally.
- Handle cases where multiple tools might need to be used in sequence before the final response is given.


In [ ]:
class Agent:
    """A tool-calling AI Agent"""

    def __init__(
        self,
        name:str = "Agent", 
        role:str = "Personal Assistant",
        instructions:str = "Help users with any question",
        model:str = "gpt-4o-mini",
        temperature:float = 0.0,
        tools:List[Tool] = [],
    ):
        self.name = name
        self.role = role
        self.instructions = instructions
        self.model = model
        self.temperature = temperature
        self.memory = Memory()
        self.memory.add_message(
            role="system",
            content=f"You're an AI Agent, your role is {self.role}, " 
                    f"and you need to {self.instructions}",
        )

        self.client = client

        self.tools = tools
        self.tool_map = {t.name:t for t in tools}
        self.openai_tools = [t.dict() for t in self.tools] if self.tools else None

    def invoke(self, user_message: str) -> str:
        self.memory.add_message(
            role="user",
            content=user_message,
        )

        ai_message = self._get_completion(
            messages = self.memory.get_messages(),
        )

        tool_calls = ai_message.tool_calls
        self.memory.add_message(
            role="assistant",
            content=ai_message.content,
            tool_calls=tool_calls,
        )

        if tool_calls:
            self._call_tools(tool_calls)
            
        return self.memory.last_message()

    def _call_tools(self, tool_calls:List[ChatCompletionMessageToolCall]):
        for t in tool_calls:
            tool_call_id = t.id
            function_name = t.function.name
            args = json.loads(t.function.arguments)
            callable_tool = self.tool_map[function_name]
            result = callable_tool(**args)
            self.memory.add_message(
                role="tool", 
                content=str(result), 
                tool_call_id=tool_call_id
            )

        ai_message = self._get_completion(
            messages = self.memory.get_messages(),
        )

        tool_calls = ai_message.tool_calls

        self.memory.add_message(
            role="assistant",
            content=ai_message.content,
            tool_calls=tool_calls,
        )

        if tool_calls:
            self._call_tools(tool_calls)


    def _get_completion(self, messages:List[Dict])-> ChatCompletionMessage:
        response = self.client.chat.completions.create(
            model=self.model,
            temperature=self.temperature,
            messages=messages,
            tools=self.openai_tools,
        )
        
        return response.choices[0].message


## 3. Build some agents and have fun

Create some specific agents with  tools, invoke then and  inspect their memory

In [ ]:
agent = Agent(
    tools=[Tool(power)]
)

In [ ]:
agent.invoke("What is 10 + 5?")

In [ ]:
agent.memory.get_messages()

In [ ]:
agent.memory.reset()

In [ ]:
agent.invoke("What is 2 to the power of 3")

In [ ]:
agent.memory.get_messages()

In [ ]:
agent.invoke("What is 3 to the power of (2 to the power of 2)?")

In [ ]:
agent.memory.get_messages()

## 4. Experiment

Now that you understood how it works, experiment with new things.

- Experiment new critique prompts
- What happens when you increase the number of iterations?
- Try accessing the memory to inspect it (agent.memory) instead of reading the outputs (verbose=False)
- What else can you try?